In [1]:
import json

import bz2
import regex
from tqdm import tqdm
from scipy import sparse

In [2]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
responses = []
with bz2.BZ2File('../5.3/banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [02:41, 1241.40it/s]


#  Домашнее задание по NLP # 2 [100 баллов] 
## Составление словарей для классификации по тональности
При классификации текстов или предложений по тональности необходимо использовать оценочные словари для предметной области, то есть, такие словари, в которых содержатся отрицательные и позитивные слова для какой-то предметной области. Идея подобных словарей основана на следующих наблюдениях: во-первых, для разных товаров используются разные оценочные слова (например бывает “захватывающая книга”, но не бывает “захватывающих лыж”), во-вторых, в контексте разных товаров одни и те же слова могут иметь разную окраску (слово “тормоз” в отзыве на велосипед имеет нейтральную окраску, в отзыве на компьютер – резко негативную, “пыль” в контексте пылесосов – нейтральную, в контексте кофемолок – положительную (“мелкий помол в пыль”)). Еще один пример: "теплое пиво" – это плохо, а "теплый свитер" – это хорошо.  

Составление таких словарей в ручную – трудоемкий процесс, но, к счастью, его не сложно автоматизировать, если собрать достаточно большие корпуса отзывов. В этом домашнем задании вам предстоит попробовать реализовать один их подходов к составлению оценочных словарей, основанный на статье Inducing Domain-Specific Sentiment Lexicons from Unlabeled Corpora (https://nlp.stanford.edu/pubs/hamilton2016inducing.pdf).


Данные для задания – уже знакомые вам отзывы на банки, собранные с нескольких сайтов Рунета. Отзывы могут быть как положительными (оценка 5), так и отрицательными (оценка 1).

In [4]:
responses[99]

{'author': 'ronnichka',
 'bank_license': 'лицензия № 880',
 'bank_name': 'Югра',
 'city': 'г. Саратов',
 'datetime': '2015-06-03 20:56:57',
 'num_comments': 0,
 'rating_grade': 3,
 'rating_not_checked': False,
 'text': 'Здравствуйте! Хотела написать, что мне месяц не выдают карту ко вкладу, ссылаясь на "нам же их из Самары везут" (на секундочку 5 часов езды от нашего города). Но! Прочитала, что людям 3,5 месяцев не выдают карту, и поняла, что у меня все хорошо, пока что. И подарок мне дали, и кулер в отделении есть. Так что я, конечно, готова ждать. Правда хотелось бы не очень долго.',
 'title': 'Карта ко вкладу'}

## Часть 1. Обучение модели word2vec [50 баллов]

1. Разбейте всю коллекцию отзывов на предложения. Лемматизируйте все слова. 
2. Обучите по коллекции предложений word2vec
3. Приведите несколько удачных и неудачных примеров решения стандартных текстов для word2vec:
    * тест на определение ближайших слов
    * тест на аналогии (мужчина – король : женщина – королева)
    * тест на определение лишнего слова.
    
4. Постройте несколько визуализаций:
    * TSNE для топ-100 (или топ-500) слов и найдите осмысленные кластеры слов
    * задайте координаты для нового пространства следующим образом: одна  ось описывает отношение "плохо – хорошо", вторая – "медленно – быстро" и найдите координаты названий банков в этих координатах.  Более формально:
    берем вектор слова "хорошо", вычитаем из него вектор слова "плохо", получаем новый вектор, который описывает разницу между хорошими и плохими словами. Берем вектор слова "сбербанк" и умножаем его на этот новый вектор – получаем координату по первой оси. Аналогично – для второй оси. Две координаты уже можно нарисовать на плоскости.  
 


Ссылка на примеры визуализаций: https://towardsdatascience.com/game-of-thrones-word-embeddings-does-r-l-j-part-2-30290b1c0b4b

In [5]:
from pymystem3 import Mystem
import re

m = Mystem()

regex = re.compile("[А-Яа-я:=!\)\()A-z\_\%/|]+")

def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text))
    except:
        return ""

def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "

In [6]:
#Импортируем имеющийся корпус стоп-слов из nltk
from nltk.corpus import stopwords

#Добавляем несколько своих стои-слов
mystopwords = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д', '-']

#Оописываем функцию удаления стоп-слов
def  remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""

In [7]:
#Разбиваем текст на слова, слова приводим к нижнему регистру, лематизируем и выбрасываем стпо-слова
fulltext = []
for response in tqdm(responses):
    
    #Приведем все слова к нижнему регистру
    r=response['text'].lower()
    #Оставляем только слова, в соответствии с шаблоном
    r=words_only(r)
    #Лематизируем слова
    r=lemmatize(r)
    #Удаляем стоп-слова
    r=remove_stopwords(r)
    #Добавляем новый отчет в список
    fulltext.append(r.split())

100%|██████████| 153499/153499 [34:11<00:00, 74.82it/s]


## Обучение модели в gensim

In [8]:
%%time
#Обучим модель word2vec от gensim
from gensim.models import Word2Vec
model = Word2Vec(fulltext, size=100, window=5, min_count=5, workers=4)
#На всякий случай, сохраняем модель в файл
model.save("sent_w2v.model")

CPU times: user 3min 59s, sys: 7.87 s, total: 4min 7s
Wall time: 2min 15s


In [9]:
#Загрузка данных из файла (на всякий случай)
#from gensim.models import Word2Vec
#model = Word2Vec.load("sent_w2v.model") 

In [10]:
#Посмотрим самые близкие слова для смайлика
model.most_similar(":)")

/home/alexey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[(':d', 0.6513125896453857),
 ('порадоваться', 0.6441833972930908),
 ('молодец!', 0.640477180480957),
 ('эх', 0.6352654695510864),
 ('держать!', 0.6349537372589111),
 ('сглазить', 0.6234007477760315),
 ('хорошо!', 0.6159003973007202),
 ('позитив', 0.6066586971282959),
 ('нга', 0.6039852499961853),
 ('супер!', 0.5995407104492188)]

In [11]:
#Для смайлика самым близким словом подобралось "хороший", что вполне корректно

In [12]:
model.most_similar("банк")

/home/alexey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('банка', 0.8396025896072388),
 ('банк!', 0.6995136737823486),
 ('банк)', 0.5935084223747253),
 ('банк:', 0.5857771635055542),
 ('сбербанк', 0.5537241101264954),
 ('хотя', 0.5020884275436401),
 ('росбанк', 0.5007374882698059),
 ('ситибанк', 0.49708041548728943),
 ('мкб', 0.4847794771194458),
 ('бм', 0.4787645637989044)]

In [13]:
#Самыми близкими оказались различные колллизии слова банк

In [14]:
model.most_similar("король")

/home/alexey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('унитаз', 0.7670700550079346),
 ('гамлет', 0.7452953457832336),
 ('овца', 0.7442737221717834),
 ('стрелять', 0.7407636046409607),
 ('праздно', 0.736426591873169),
 ('публика', 0.7356441020965576),
 ('стадо', 0.7354274392127991),
 ('платочек', 0.7332589626312256),
 ('шнурок', 0.72869873046875),
 ('галстучек', 0.7264435887336731)]

In [15]:
#В данном контексте, прямо как пример изменения смысла слова. В отзывах о банках слово король имеет явно негативный 
#оттенок и употребляется скорее в саркостическом смысле

In [16]:
model.most_similar("королева")

/home/alexey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('серпухов', 0.87547367811203),
 ('пятигорск', 0.8674222826957703),
 ('ломоносов', 0.8632595539093018),
 ('реутов', 0.8553909063339233),
 ('балашиха', 0.8532413244247437),
 ('академик', 0.8502539992332458),
 ('павловский', 0.8485872149467468),
 ('мытищи', 0.8483651876449585),
 ('петрозаводск', 0.8463224768638611),
 ('волоколамский', 0.8452526330947876)]

In [17]:
#Видимо королеву считают городом Королев и ищутся близкие слова

In [18]:
model.most_similar("карта")

/home/alexey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('карточка', 0.8929780721664429),
 ('карта!', 0.8205209970474243),
 ('карта)', 0.7202193140983582),
 ('карта:', 0.6846864223480225),
 ('кредитка', 0.6567911505699158),
 ('неименной', 0.6256096363067627),
 ('счет', 0.5968410968780518),
 ('именной', 0.5908063054084778),
 ('дебетовый', 0.5830533504486084),
 ('карта!!!', 0.5806609392166138)]

In [19]:
#В данном случае слова выглядят вполне разумно

In [20]:
model.most_similar(positive=["карта", "отделение"], negative=["удобно"])

/home/alexey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('карта!', 0.5380128026008606),
 ('карточка', 0.5022581815719604),
 ('офис', 0.49057281017303467),
 ('вновь', 0.4821704030036926),
 ('повторно', 0.4777492582798004),
 ('перевыпущенный', 0.4737043082714081),
 ('осб', 0.4704963266849518),
 ('филиал', 0.4635285437107086),
 ('моментум', 0.44419583678245544),
 ('отд', 0.42208269238471985)]

In [21]:
#Честно говоря, ожидал, что в таком контексте самым близким словом будет слово "плохо"

In [22]:
model.most_similar(positive=["сбербанк","тинькоф"], negative=["хороший"])

/home/alexey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('приватбанк', 0.6089859008789062),
 ('ситибанк', 0.5843981504440308),
 ('ржд', 0.544487476348877),
 ('альфабанка', 0.5349779725074768),
 ('xxxxxxxxxxx', 0.5261602997779846),
 ('(сбербанк)', 0.5236098766326904),
 ('viza', 0.5095403790473938),
 ('citibank', 0.5018184781074524),
 ('промсвязьбанк', 0.496534526348114),
 ('альфабанк', 0.49026113748550415)]

In [23]:
model.most_similar(positive=["король", "женщина"], negative=["мужчина"])

/home/alexey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('молодежь', 0.7142486572265625),
 ('унитаз', 0.6977387070655823),
 ('погубить', 0.6889057755470276),
 ('пенсионер!', 0.6879324913024902),
 ('стадо', 0.6815958023071289),
 ('овца', 0.6787269115447998),
 ('коллектив!', 0.6770848631858826),
 ('мальчишка', 0.6765292286872864),
 ('человек!!!', 0.6721256971359253),
 ('клиентоориентированность!', 0.6647109389305115)]

In [24]:
model.doesnt_match("борщ сметана макароны пирожок консомэ кошка".split())

/home/alexey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'борщ'

In [25]:
#Вообще говоря, результат не правильный, ожидалось, что лишней будет "кошка"

In [26]:
model.doesnt_match("мкб сбербанк втб тинькоф карта".split())

/home/alexey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'карта'

In [27]:
#Интересный момент. Если написать "Тиньков", то словарь, видимо не ассоциирует это слово с банком и выкидывает именно его

### Визуализация пространства слов 

In [28]:
top_words = []
from nltk import FreqDist

#Формируем словарь частотности слов
fd = FreqDist()
for text in fulltext:
    fd.update(text)

#Сохраняем 500 наиболее часто употребляемых
for i in fd.most_common(500):
    top_words.append(i[0])

#Выводим топ 500 слов на печать
print(top_words)

['банк', 'карта', 'деньги', 'день', 'кредит', 'который', 'весь', 'отделение', 'сотрудник', 'клиент', 'мочь', 'счет', 'свой', 'сказать', 'сумма', 'получать', 'заявление', 'год', 'время', 'вопрос', 'кредитный', 'приходить', 'вклад', 'офис', 'данный', 'телефон', 'ваш', 'платеж', 'договор', 'звонить', 'ответ', 'номер', ')', 'банкомат', 'сбербанк', 'очень', 'позвонить', 'написать', 'г', 'обращаться', 'месяц', 'решать', 'проблема', 'средство', 'работать', 'работа', 'рубль', 'документ', 'звонок', 'ситуация', 'отвечать', 'хотеть', 'почему', 'человек', 'операция', 'давать', 'говорить', 'информация', 'сообщать', 'сделать', 'оформлять', 'смс', 'срок', 'услуга', 'просто', 'должный', 'заявка', 'банка', 'знать', 'обслуживание', 'очередь', ':', 'сегодня', 'минута', 'первый', 'девушка', 'ждать', 'руб', 'сайт', 'претензия', 'новый', 'ничто', 'просить', 'интернет', 'открывать', 'принимать', 'снимать', 'процент', 'выдавать', 'никакой', 'являться', 'неделя', 'оператор', 'становиться', 'понимать', '%', 'пр

In [29]:
top_words_vec = model[top_words]

/home/alexey/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [30]:
#При помощи TSNE понижаем размерность до 2-х компонент, чтобы можно было нарисовать пространство слов
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
top_words_tsne = tsne.fit_transform(top_words_vec)

In [31]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

In [32]:
#На визуализации видно, что радом расположены слова "Снятие", "снимать", "вносить", "полагать", "переводить", "пополнять"
#"поступать", "зачисление", "зачислять", "начислять", "возврат" - все их можно отнести к кластеру Работа с банковской картой
#Другой кластер - "номер", "телефон", "колл", "телефонный", "трубка". Можно их обозначить как Обращения по телефону
#Еще один кластер - "тысяча", "рубль", "руб", "размер", "сумма". Можно их обозначить как Информация о деньгах

## Часть 2. Распространение метки [50 баллов]

Определите 5-8 позитивных слов (например, “быстрый”, “удобный”) и 5-8  негативных слов (например,“очередь”, “медленно”). Эти слова будут основной будущего оценочного словаря. Пусть позитивному классу соответствует метка 1, негативному – -1. Пометьте выбранные слова в лексическом графе соответствующими метками. Запустите любой известный вам метод распространения метки (Label Propogation) в лексическом графе. На выходе метода распространения ошибки должны быть новые слова, помеченные метками 1 и -1 – это и есть искомые оценочные слова.

Алгоритмы распространения метки устроены примерно так: пусть мы находимся в выршине, помеченном +1. С какой-то вероятностью мы переносим эту метку на соседние узлы. С меньшей вероятностью переносим ее на вершины на расстоянии два. В конце распространения метки, часть вершин оказывается помечена меткой +1, часть – -1, большая часть остается без метки.

Рекомендуемые алгоритмы распространения метки:
1. ```graphlab.label_propagation``` (```graphlab``` доступен бесплатно по образовательной лицензии)
2. ```sklearn.semi_supervised.LabelPropagation``` 
3. ```sklearn.semi_supervised.LabelSpreading```

In [40]:
# пример построения графа 

import jgraph as ig
g = ig.Graph(directed=True)
for word in model.wv.vocab.keys():
    g.add_vertex(word)
    
    
    
for word in model.wv.vocab.keys() :
    node = g.vs.select(name = word).indices[0]
    similar_words = model.most_similar(word, topn=5)
    for sim in similar_words:
        word1 = sim[0]
        val  = sim[1]
        new_node = g.vs.select(name = word1).indices[0]
        g.add_edge(node, new_node, weight = val)

AttributeError: module 'jgraph' has no attribute 'Graph'